In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

plt.style.use("dark_background")
warnings.filterwarnings("ignore")

In [ ]:
ds = pd.read_csv("/share/dutta/eyao/dataset/kaggle/spaceship-titanic/train.csv")

ds

In [ ]:
ts = pd.read_csv("/share/dutta/eyao/dataset/kaggle/spaceship-titanic/test.csv")

ss = pd.read_csv("/share/dutta/eyao/dataset/kaggle/spaceship-titanic/sample_submission.csv")

ts = pd.merge(ts, ss, on='PassengerId')

ts

In [ ]:
ds.info()

In [ ]:
ds.isnull().sum()

In [ ]:
ds.isna().sum()

In [ ]:
ds.describe().T

In [ ]:
ds.drop(columns=["PassengerId", "Name"], axis=1, inplace=True)
ts.drop(columns=["PassengerId", "Name"], axis=1, inplace=True)

In [ ]:
spe = ds

In [ ]:
spe.dropna(inplace=True)

numCol = spe.select_dtypes('number').columns

spe

In [ ]:
fig, ax = plt.subplots(4, 3, figsize=(20, 15))

for idx, (col, ax) in enumerate(zip([*numCol, *numCol], ax.flatten())):
    if len(numCol) <= idx:
        sns.boxplot(y=spe[col], ax=ax)
    else:
        sns.kdeplot(x=spe[col], ax=ax, fill=True)

In [ ]:
from sklearn.preprocessing import LabelEncoder

for col in ds.select_dtypes(include=[object, bool]):
    ds[col] = LabelEncoder().fit_transform(ds[col])
    
for col in ts.select_dtypes(include=[object, bool]):
    ts[col] = LabelEncoder().fit_transform(ts[col])
    
ds

In [ ]:
ds.isnull().sum()

In [ ]:
for col in ds.columns:
    ds[col].fillna = np.median(ds[col])
    
for col in ts.columns:
    ts[col].fillna = np.median(ts[col])

ds

In [ ]:
ds.Transported.value_counts()

In [ ]:
from sklearn.preprocessing import MinMaxScaler

y = ds.Transported.values

ds = pd.DataFrame(MinMaxScaler().fit_transform(ds), columns=ds.columns)

ds.Transported = y

y = ts.Transported.values

ts = pd.DataFrame(MinMaxScaler().fit_transform(ts), columns=ts.columns)

ts.Transported = y

ds

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_val, y_train, y_val = train_test_split(ds.iloc[:, :-1], ds.iloc[:, -1], train_size=.8, random_state=10)

x_train.shape, x_val.shape, y_train.shape, y_val.shape

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

model = Sequential([
    Dense(x_train.shape[1], activation="relu"),
    Dense(24, activation="relu"),
    Dense(12, activation="relu"),
    Dense(10, activation="relu"),
    Dense(1, activation="sigmoid")
])

model.compile(loss="binary_crossentropy", optimizer=Adam(.03), metrics=["Accuracy"])

history = model.fit(x_train, y_train, epochs=50, validation_batch_size=64, validation_data=(x_val, y_val))

In [ ]:
loss = history.history['loss']
acc = history.history['Accuracy']

fig, ax = plt.subplots(1, 2, figsize=(20, 5))

ax[0].plot(acc)
ax[0].set_xlabel("Accuracy")

ax[1].plot(loss)
ax[1].set_xlabel("Loss")

In [ ]:
loss = history.history['val_loss']
acc = history.history['val_Accuracy']

fig, ax = plt.subplots(1, 2, figsize=(20, 5))

ax[0].plot(acc)
ax[0].set_xlabel("Accuracy")

ax[1].plot(loss)
ax[1].set_xlabel("Loss")

In [ ]:
yPred = model.predict(ts.iloc[:, :-1])

yPred = pd.Series(np.ravel(yPred) < .5)

yPred.value_counts()

In [ ]:
ss["Transported"] = yPred

ss

In [ ]:
ss.to_csv('submission.csv', index = False)

Peace ✌✌